<a href="https://colab.research.google.com/github/brianp0513/NLP-project/blob/main/Hybrid_NLP_Project_version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# data preprocessing

In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
%matplotlib inline
import time
import re
from wordcloud import WordCloud
import random
import os
from sklearn import metrics 
from scipy.spatial.distance import cdist 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
  
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from collections import OrderedDict
from sklearn.mixture import GaussianMixture
import os
plt.style.use('ggplot')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def getTextFromFiles(df, data_path, depression, limit):
    """Return Data Frame """

    for file in os.listdir(data_path)[:limit]:
        with open(data_path + "/" + file, 'r', encoding="ISO-8859-1") as file1:
            file1 = file1.read()
            df = df.append({'text': file1, 'depression': int(depression)}, ignore_index=True)

    return df


In [5]:
def dataPreprocessingForX(df, columnName1):
    df[columnName1] = df[columnName1].map(lambda text: text.lower())
    df[columnName1] = df[columnName1].map(lambda text: nltk.tokenize.word_tokenize(text))
    stop_words = set(nltk.corpus.stopwords.words('english'))
    df[columnName1] = df[columnName1].map(lambda tokens: [w for w in tokens if not w in stop_words])
    df[columnName1] = df[columnName1].map(lambda text: ' '.join(text))
    df[columnName1] = df[columnName1].map(lambda text: re.sub('[^A-Za-z]+', ' ', text))
    df[columnName1] = df[columnName1].map(lambda text: nltk.tokenize.word_tokenize(text))
    lemmatizer = WordNetLemmatizer()
    df[columnName1] = df[columnName1].map(lambda text: [lemmatizer.lemmatize(i) for i in text])
    df[columnName1] = df[columnName1].map(lambda text: ' '.join(text))

def dataPreprocessingForY(df, columnName2):
    df[columnName2] = df[columnName2].astype('int32')

In [99]:
data_path_d = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_depression"
data_path_nd = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_non_depression"
df = pd.DataFrame(columns=['text', 'depression'])
df = getTextFromFiles(df, data_path_d, 1, 1000)
df = getTextFromFiles(df, data_path_nd, 0, 500)


In [100]:
dataPreprocessingForX(df, 'text')
dataPreprocessingForY(df, 'depression')

In [133]:
X = df['text'].to_numpy()
Y = df['depression'].to_numpy()
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.30)

max_words = 1000
max_len = 150

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)


# classifier.fit(counts, Y_train)
# test_counts = count_vectorizer.transform(X_test)
# predictions = classifier.predict(test_counts)

# K mean

In [102]:

def kmean(X_test_kmean):
  init_centroid = 'k-means++'
  y_kmList = []
  distortions = [] 
  inertias = [] 
  mapping1 = {} 
  mapping2 = {}
  K = range(1,10)
  # X_train_kmean =  test_sequences_matrix
  # X_train_kmean = counts.toarray()
  for k in K:
      #Building and fitting the model 
      kmeanModel = KMeans(n_clusters=k, init = init_centroid, random_state=0)
      y_km = kmeanModel.fit(Xtest_kmean)  
        
      distortions.append(sum(np.min(cdist(X_test_kmean, kmeanModel.cluster_centers_, 
                        'euclidean'),axis=1)) / X_test_kmean.shape[0]) 
      inertias.append(kmeanModel.inertia_) 

      y_kmList.append(y_km.labels_)

      mapping1[k] = sum(np.min(cdist(X_test_kmean, kmeanModel.cluster_centers_, 
                  'euclidean'),axis=1)) / X_test_kmean.shape[0] 
      mapping2[k] = kmeanModel.inertia_ 


  plt.plot(K, distortions, 'bx-') 
  plt.xlabel('Values of K') 
  plt.ylabel('Distortion') 
  plt.title('The Elbow Method using Distortion') 
  plt.show() 

  # for key,val in mapping1.items(): 
  #   print(str(key)+' : '+str(val))
  
  # plt.plot(K, inertias, 'bx-')
  # plt.xlabel('Values of K')
  # plt.ylabel('Inertia')
  # plt.title('The Elbow Method using Inertia') 
  # plt.show() 

  # for key,val in mapping2.items(): 
  #   print(str(key)+' : '+str(val))


  return y_kmList
# print(np.shape(y_km))
# plt.scatter(X[y_km==0, 0], X[y_km==0,1],c='lightgreen',marker='s',s=510,label='cluster1')
# plt.scatter(X[y_km==1, 0], X[y_km==1,1],c='orange',marker='o',s=50,label='cluster2')
# plt.scatter(km.cluster_centers_[:,0],km.cluster_centers_[:,1],c='red',marker='*',s=50,label='center')

# plt.legend()
# plt.grid(True)
# plt.show()


In [104]:
def kmeanClusterResult(y_kmList, X):
  zero_list = []
  one_list = []
  two_list = []

  for idx,iteration in enumerate(y_kmList[1]):
    if iteration == 1:
      one_list.append(idx)
    elif iteration == 2:
      two_list.append(idx)
    else:
      zero_list.append(idx)
  # print(zero_list)
  # print(one_list)
  # print(two_list)
  # print(X.shape)
  cluster_zero = []
  cluster_one = []
  cluster_two = []
  for index in zero_list:
    sentence = X[index]
    cluster_zero.append(sentence)
    
  # print(cluster_zero)
  for index in one_list:
    sentence = X[index]
    cluster_one.append(sentence)

  for index in two_list:
    sentence = X[index]
    cluster_two.append(sentence)

  return zero_list, one_list

# Gaussian Mixture

In [116]:
def gmm(X_test_gmm):
  y_gmmList = []

  # X =  sequences_matrix
  #Building and fitting the model 
  gmm = GaussianMixture(n_components = 2)
  gmm.fit(X_test_gmm)
  y_gmm = gmm.predict(X_test_gmm)
  y_gmmList.append(y_gmm)

  return y_gmmList

def gmmClusterResult(y_gmmList, X):
  zero_list = []
  one_list = []
  two_list = []

  for idx,iteration in enumerate(y_gmmList[0]):
    if iteration == 1:
      one_list.append(idx)
    elif iteration == 2:
      two_list.append(idx)
    else:
      zero_list.append(idx)
  # print(zero_list)
  # print(one_list)
  # print(two_list)
  # print(X.shape)
  cluster_zero = []
  cluster_one = []
  cluster_two = []
  for index in zero_list:
    sentence = X[index]
    cluster_zero.append(sentence)
    
  # print(cluster_zero)
  for index in one_list:
    sentence = X[index]
    cluster_one.append(sentence)

  for index in two_list:
    sentence = X[index]
    cluster_two.append(sentence)

  return zero_list, one_list

In [113]:
# print(sequences_matrix)
# print(y_kmList)

In [114]:
def makeWorldCloud(llist):
  depression_words = ''.join(llist)
  depression_wordclod = WordCloud(width = 512,height = 512).generate(depression_words)
  plt.figure(figsize = (10, 8), facecolor = 'k')
  plt.imshow(depression_wordclod)
  plt.axis('off')
  plt.tight_layout(pad = 0)
  plt.show()
# makeWorldCloud(cluster_zero)
# makeWorldCloud(cluster_one)
# makeWorldCloud(cluster_two)

In [118]:
# y_kmList = kmean(sequences_matrix)
# zero_list, one_list = kmeanClusterResult(y_kmList, X)
y_gmmList = gmm(sequences_matrix)
zero_list, one_list = gmmClusterResult(y_gmmList, X)

# LSTM

In [138]:
# X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

max_words = 1000
max_len = 150

# tok = Tokenizer(num_words=max_words)
# tok.fit_on_texts(X_train)
# sequences = tok.texts_to_sequences(X_train)
# sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
# test_sequences = tok.texts_to_sequences(X_test)
# test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

sequences_zeroClust = tok.texts_to_sequences(X[zero_list])
sequences_matrix_zeroClust = sequence.pad_sequences(sequences_zeroClust,maxlen=max_len)
sequences_oneClust = tok.texts_to_sequences(X[one_list])
sequences_matrix_oneClust = sequence.pad_sequences(sequences_oneClust,maxlen=max_len)

test_sequences_matrix = sequences_matrix_zeroClust
Y_test_LSTM = Y[zero_list]

# test_sequences_matrix = sequences_matrix_oneClust
# Y_test = Y[one_list]

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_words,50,input_length=max_len))
model.add(tf.keras.layers.LSTM(64))
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))

# model.build(input_shape=sequences_matrix.shape)
# model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

model.fit(sequences_matrix.astype(float), Y_train.astype(float), batch_size=4,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
accr = model.evaluate(test_sequences_matrix, Y_test_LSTM.astype(float))

print(" LSTM")
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))



Epoch 1/10
210/210 [==============================] - 2s 11ms/step - loss: 0.6431 - accuracy: 0.6464 - val_loss: 0.5523 - val_accuracy: 0.6762
Epoch 2/10
210/210 [==============================] - 2s 9ms/step - loss: 0.3991 - accuracy: 0.8357 - val_loss: 0.3099 - val_accuracy: 0.8667
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.1846 - accuracy: 0.9286
 LSTM
Test set
  Loss: 0.185
  Accuracy: 0.929


# Naive Bayes

In [139]:
classifier = MultinomialNB()
# count_vectorizer = CountVectorizer(ngram_range = ngram_range, min_df = min_df)
# counts = count_vectorizer.fit_transform(X_train)
# featnames = count_vectorizer.get_feature_names()
# classifier.fit(counts, Y_train)

ngram_range = (1,5)
min_df = 0.1
count_vectorizer = CountVectorizer(ngram_range = ngram_range, min_df = min_df)
counts = count_vectorizer.fit_transform(X_train)

# featnames = count_vectorizer.get_feature_names()
classifier.fit(counts, Y_train)
print(X_test.shape)
test_counts = count_vectorizer.transform(X_test)
# X[zero_list]
# y_kmList = kmean(test_counts.toarray())
predictions = classifier.predict(test_counts)
print("Accuracy: ",(predictions == Y_test).mean())
# y_kmList = kmean(counts.toarray())
# zero_list, one_list = kmeanClusterResult(y_kmList, X)

y_gmmList = gmm(test_counts.toarray())
zero_list, one_list = gmmClusterResult(y_gmmList, X)

print(zero_list)
print(one_list)


test_counts = count_vectorizer.transform(X[zero_list])
# predictions = classifier.predict(test_counts[zero_list])
predictions = classifier.predict(test_counts)

# Y_test = Y[zero_list]

print(predictions)
# Y[one_list]
print("Accuracy: ",(predictions == Y[zero_list]).mean())

test_counts = count_vectorizer.transform(X[one_list])
# predictions = classifier.predict(test_counts[zero_list])
predictions = classifier.predict(test_counts)
# Y_test = Y[one_list]
print(predictions)
# Y[one_list]
print("Accuracy: ",(predictions == Y[one_list]).mean())


(450,)
Accuracy:  0.8711111111111111
[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 124, 125, 126, 129, 132, 135, 136, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 171, 173, 174, 175, 176, 177, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 213, 214, 215, 216, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 240, 243, 244, 245, 246, 24